# <center>*Segmenting and Clustering Neighborhoods in Toronto*

## Part 1. Scrapping the Wikipedia page. Wrangling the data, cleaning, and reading it into a pandas dataframe into a structured format

### Download all the dependencies

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import json # library to handle JSON files
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

! pip install folium==0.5.0
import folium # plotting library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 79 kB 10.6 MB/s eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=0e9202b0be21736d6b613c8ffe7317008c4245aa98aa69e2c25ab3f497543c43
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Libraries imported.


### Scrapping the Wikipedia page. Wrangling the data, cleaning, and reading it into a pandas dataframe into a structured format¶

In [2]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 

soup = BeautifulSoup(data, 'html5lib')

# creating Toronto Dataframe
df= pd.DataFrame(columns = ['PostalCode','Borough','Neighborhood'])


# scrapping all rows of the table
table_rows = soup.find('table').tbody.find_all('tr')

# filtering rows and inserting data to df_toronto
for rows in table_rows :
    for column in rows.find_all('td') :
        if column.span.text != 'Not assigned' :
            span  = column.span.text.split('(')
            df = df.append({'PostalCode' : column.b.text,
                              'Borough' : span[0],
                              'Neighborhood' : span[1][:-1]}, ignore_index=True)
            
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df = df.sort_values('PostalCode').reset_index(drop = True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [3]:
# Check the size of dataframe
df.shape

(103, 3)

## Part 2. Segmenting Neighborhoods in Toronto

### Download and explore dataset

In [10]:
!wget -O GeoSpataial_Data https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv

--2021-06-26 03:07:42--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2788 (2.7K) [text/csv]
Saving to: ‘GeoSpataial_Data’

GeoSpataial_Data    100%[===================>]   2.72K  --.-KB/s    in 0s      

2021-06-26 03:07:42 (39.7 MB/s) - ‘GeoSpataial_Data’ saved [2788/2788]



### Getting the Latitude and Longitude of each Postal_code

In [11]:
df_geospatial = pd.read_csv('GeoSpataial_Data')
df_geospatial.columns = ['PostalCode','Latitude', 'Longitude']
df_geospatial.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# Merge 2 dataframes
df_merge = pd.merge(df, df_geospatial, on= 'PostalCode')
df_merge.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848


### Use geopy library to get the latitude and longitude values of Toronto

In [13]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to the map
for lat, lng, borough, neighborhood in zip(
        df_merge['Latitude'], 
        df_merge['Longitude'], 
        df_merge['Borough'], 
        df_merge['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto


### Simplify the Toronto map and segment and cluster only the neighborhoods in Scarborough Borough¶

In [15]:
#create a new dataframe of the Scarborough data.
scarborough_data = df_merge[df_merge['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848


In [55]:
# Get the geographical coordinates of Scarborough.
address = 'Scarborough, Toronto, Ontario'

geolocator = Nominatim(user_agent="scarborough_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.7729744, -79.2576479.


In [16]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

### Define Foursquare Credentials and Version

In [17]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: IN51BRT2QGA0SLEQQAVZXDFUP1GFL1QWUCDFT1KYUCLTYTEJ
CLIENT_SECRET:2BQWKAXKAWRDLPFVFVKXJTYZSHMGATVY5DZCFNHCYNXNSTPW


### Let's explore the first neighborhood in our dataframe

In [18]:
scarborough_data.loc[0, 'Neighborhood']

'Malvern / Rouge'

### Get the neighborhood's latitude and longitude values.

In [19]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern / Rouge are 43.806686299999996, -79.19435340000001.


###  Define the corresponding URL and get request

In [20]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json() 
results

{'meta': {'code': 200, 'requestId': '60d69d37ace81a7d8c72492c'},
 'response': {'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 18,
  'suggestedBounds': {'ne': {'lat': 43.81568630900001,
    'lng': -79.18190576146081},
   'sw': {'lat': 43.797686290999984, 'lng': -79.20680103853921}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c6ac7de35d3be9a50bf2206',
       'name': 'RBC Royal Bank',
       'location': {'address': '865 MILNER AVE',
        'crossStreet': 'Morningside',
        'lat': 43.79878248056552,
        'lng': -79.19709031445504,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.79878248056552,
          'lng': -79.19709031445504}],
        'distance': 

### Get relevant part of JSON and transform it into a pandas dataframe

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
#Clean the json and structure it into a pandas datafram
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.798782,-79.197090
1,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
2,Wendy's,Fast Food Restaurant,43.802008,-79.198080
3,Harvey's,Restaurant,43.800020,-79.198307
4,Caribbean Wave,Caribbean Restaurant,43.798558,-79.195777
5,Staples Morningside,Paper / Office Supplies Store,43.800285,-79.196607
6,Lee Valley,Hobby Shop,43.803161,-79.199681
7,Bus Stop: 85 & 116,Bus Station,43.802198,-79.199389
8,Tim Hortons / Esso,Coffee Shop,43.801863,-79.199296
9,Ecopainting inc.,Construction & Landscaping,43.808417,-79.202392


In [23]:
# Number of venvues were returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

18 venues were returned by Foursquare.


### Explore Neighborhoods in Scarborough

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Code to run the above function on each neighborhood and create a new dataframe called scarborough_venues.

In [25]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )
print(scarborough_venues.shape)
scarborough_venues.head()

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt)
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
(84, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Guildwood / Morningside / West Hill,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
3,Guildwood / Morningside / West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Guildwood / Morningside / West Hill,43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant


In [26]:
#Check how many venues were returned for each neighborhood and unique categories can be curated from all the returned venues
scarborough_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 50 uniques categories.


### Analyze Each Neighborhood

In [27]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Discount Store,Donut Shop,Electronics Store,Fast Food Restaurant,Gaming Cafe,Gas Station,General Entertainment,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Supermarket,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Malvern / Rouge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Rouge Hill / Port Union / Highland Creek,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Guildwood / Morningside / West Hill,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Guildwood / Morningside / West Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Guildwood / Morningside / West Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [28]:
# Comfirm the size
scarborough_onehot.shape

(84, 51)

### let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped

,Neighborhood,American Restaurant,Arts & Crafts Store,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Discount Store,Donut Shop,Electronics Store,Fast Food Restaurant,Gaming Cafe,Gas Station,General Entertainment,Hakka Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean BBQ Restaurant,Latin American Restaurant,Lounge,Medical Center,Mexican Restaurant,Middle Eastern Restaurant,Motel,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Supermarket,Thai Restaurant,Train Station,Vietnamese Restaurant
0,Agincourt),0.0,0.00,0.000000,0.000000,0.000000,0.0,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.25,0.25,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,Birch Cliff / Cliffside West,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
2,Cedarbrae,0.0,0.00,0.142857,0.142857,0.142857,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.142857,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.142857,0.000000,0.000000
3,Clarks Corners / Tam O'Shanter / Sullivan,0.0,0.00,0.000000,0.000000,0.100000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.100000,0.100000,0.00,0.000000,0.000000,0.000000,0.200000,0.000000,0.100000,0.00,0.000000,0.000000,0.000000,0.000000,0.1,0.00,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.200000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.100000,0.000000,0.000000
4,Cliffside / Cliffcrest / Scarborough Village West,0.5,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.00,0.000000,0.000000,0.00,0.5,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
5,Dorset Park / Wexford Heights / Scarborough To...,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.333333,0.000000,0.0,0.00,0.00,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.166667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.166667
6,Golden Mile / Clairlea / Oakridge,0.0,0.00,0.000000,0.222222,0.000000,0.0,0.000000,0.222222,0.111111,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.111111,0.000000,0.111111,0.0,0.00,0.00,0.00,0.000000,0.000000,0.00,0.0,0.111111,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.111111,0.0,0.000000,0.000000,0.000000,0.000000
7,Guildwood / Morningside / West Hill,0.0,0.00,0.000000,0.000000,0.111111,0.0,0.111111,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.111111,0.111111,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.111111,0.0,0.00,0.00,0.00,0.111111,0.111111,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.111111,0.111111,0.000000,0.00,0.00,

In [30]:
# Comfirm the size
scarborough_grouped.shape

(16, 51)

### Print each neighborhood along with the top 5 most common venues

In [31]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt)----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4        American Restaurant  0.00


----Birch Cliff / Cliffside West----
                   venue  freq
0        College Stadium  0.25
1  General Entertainment  0.25
2           Skating Rink  0.25
3                   Café  0.25
4    American Restaurant  0.00


----Cedarbrae----
                venue  freq
0  Athletics & Sports  0.14
1              Bakery  0.14
2                Bank  0.14
3     Thai Restaurant  0.14
4    Hakka Restaurant  0.14


----Clarks Corners / Tam O'Shanter / Sullivan----
                  venue  freq
0           Pizza Place   0.2
1  Fast Food Restaurant   0.2
2    Italian Restaurant   0.1
3       Thai Restaurant   0.1
4                  Bank   0.1


----Cliffside / Cliffcrest / Scarborough Village West----
                       venue  freq
0        American Restaurant   

### Sort the venues in descending order

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create the new dataframe and display the top 10 venues for each neighborhood

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt),Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store
1,Birch Cliff / Cliffside West,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store
2,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Caribbean Restaurant,Coffee Shop,General Entertainment,Gaming Cafe
3,Clarks Corners / Tam O'Shanter / Sullivan,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Thai Restaurant,Gas Station,Italian Restaurant,Bank,Coffee Shop,Vietnamese Restaurant,Gaming Cafe
4,Cliffside / Cliffcrest / Scarborough Village West,American Restaurant,Motel,Chinese Restaurant,General Entertainment,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store,Donut Shop,Discount Store


## Part 3. Cluster Neighborhoods

### Run k-means to cluster the neighborhood into 5 clusters.

In [34]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 0, 1, 1, 1, 3, 4], dtype=int32)

### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

scarborough_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,4,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,General Entertainment,Gas Station,Gaming Cafe,Electronics Store,Donut Shop,Discount Store
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,2,Bar,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,General Entertainment,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store,Donut Shop
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,1,Intersection,Bank,Electronics Store,Breakfast Spot,Donut Shop,Restaurant,Rental Car Location,Medical Center,Mexican Restaurant,Vietnamese Restaurant
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Vietnamese Restaurant,Ice Cream Shop,General Entertainment,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Caribbean Restaurant,Coffee Shop,General Entertainment,Gaming Cafe


#### Visualize the resulting clusters

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'],
                                  scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= rainbow[(cluster)-1],
        fill=True,
        fill_color= rainbow[(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [37]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,0,American Restaurant,Motel,Chinese Restaurant,General Entertainment,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store,Donut Shop,Discount Store


#### Cluster 2

In [38]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,1,Intersection,Bank,Electronics Store,Breakfast Spot,Donut Shop,Restaurant,Rental Car Location,Medical Center,Mexican Restaurant,Vietnamese Restaurant
4,Scarborough,1,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Gas Station,Caribbean Restaurant,Coffee Shop,General Entertainment,Gaming Cafe
5,Scarborough,1,Spa,Playground,Vietnamese Restaurant,Caribbean Restaurant,General Entertainment,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store,Donut Shop
7,Scarborough,1,Bakery,Bus Line,Ice Cream Shop,Intersection,Soccer Field,Bus Station,Park,Coffee Shop,General Entertainment,Gas Station
9,Scarborough,1,General Entertainment,Skating Rink,College Stadium,Café,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store
10,Scarborough,1,Indian Restaurant,Pet Store,Chinese Restaurant,Gaming Cafe,Vietnamese Restaurant,Soccer Field,General Entertainment,Supermarket,Fast Food Restaurant,Electronics Store
11,Scarborough,1,Bakery,Smoke Shop,Sandwich Place,Middle Eastern Restaurant,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store
12,Scarborough,1,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store
13,Scarborough,1,Fast Food Restaurant,Pizza Place,Chinese Restaurant,Thai Restaurant,Gas Station,Italian Restaurant,Bank,Coffee Shop,Vietnamese Restaurant,Gaming Cafe
14,Scarborough,1,Arts & Crafts Store,Intersection,Playground,Park,Vietnamese Restaurant,Chinese Restaurant,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store


#### Cluster 3

In [39]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2,Bar,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,General Entertainment,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store,Donut Shop


#### Cluster 4

In [40]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,3,Coffee Shop,Korean BBQ Restaurant,Mexican Restaurant,Vietnamese Restaurant,Ice Cream Shop,General Entertainment,Gas Station,Gaming Cafe,Fast Food Restaurant,Electronics Store
6,Scarborough,3,Discount Store,Coffee Shop,Train Station,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,General Entertainment,Gas Station,Gaming Cafe,Fast Food Restaurant


#### Cluster 5

In [41]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4,Fast Food Restaurant,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,General Entertainment,Gas Station,Gaming Cafe,Electronics Store,Donut Shop,Discount Store
